**Preprecessing steps Experimentation**:
- After perfroming several experiments I got the best possible results
- I have also implement some other models such as Decision trees, Random forest, But their performance is very. So, I've decided to remove them.
- I have tried implementing pca but it also doesn't make significant improvements, sometimes it degrades the performance as well.
- I have tried implementing lbp but it also degrading the performance of model sometimes so i dediced to remove it.

In [1]:
import numpy as np 
import os
from skimage.feature import hog, local_binary_pattern
from skimage import color
import cv2
from sklearn.preprocessing import LabelEncoder

'''def extract_combined_features(image):
    hog_feat = hog(image,pixels_per_cell=(8,8),cells_per_block=(2,2),visualize=False)
    
    lbp = local_binary_pattern(image, 8, 1, method='uniform')
    lbp_hist,_ = np.histogram(lbp.ravel(),bins=np.arange(0, 11), density=True)
    
    return np.concatenate([hog_feat, lbp_hist])'''
def extract_features_from_folder(folder_path):
    features = []
    labels = []

    for label in os.listdir(folder_path):
        class_path = os.path.join(folder_path, label)
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img = cv2.resize(img, (128, 128))
            gray = color.rgb2gray(img)

            hog_feat = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
            features.append(hog_feat)
            labels.append(label)

    X = np.array(features)
    y = np.array(labels)

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    return X, y_encoded, le

c:\Users\Public\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
from preprocessing import extract_features_from_folder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [3]:
folder_path = 'cropped/train'
X_raw, y_encoded, le = extract_features_from_folder(folder_path)

# PCA


In [12]:
pca = PCA(n_components=100)  # Try 50-100
X_reduced = pca.fit_transform(X_raw)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_raw, y_encoded, test_size=0.2,random_state=0)

In [5]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf',C=5,probability=True))])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.7410071942446043

In [6]:
print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         6
           1       0.69      0.90      0.78        10
           2       0.80      0.67      0.73         6
           3       1.00      0.67      0.80         3
           4       0.33      0.33      0.33         3
           5       0.67      0.50      0.57         4
           6       0.70      0.88      0.78         8
           7       0.60      0.60      0.60         5
           8       0.91      1.00      0.95        10
           9       1.00      0.80      0.89         5
          10       0.93      1.00      0.96        13
          11       0.67      0.50      0.57         4
          12       0.46      0.86      0.60         7
          13       1.00      0.62      0.77         8
          14       0.50      0.57      0.53         7
          15       1.00      0.56      0.71         9
          16       0.67      0.57      0.62         7
          17       0.83    

In [7]:
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression 
model_params = {
    'svm':{
        'model':SVC(probability=True),
        'params':{
            'svc__C': [1, 10, 100],
            'svc__kernel': ['linear', 'rbf'],
        }
    },
    'logistic':{
        'model':LogisticRegression(),
        'params':{
            'logisticregression__C': [1, 5, 10],
        }
    }
}

In [8]:
from sklearn.pipeline import make_pipeline

scores = []
best_estimators = {}
import pandas as pd
for algo, mp in model_params.items():
    pipe = make_pipeline(StandardScaler(),mp['model'])
    clf = GridSearchCV(pipe, mp['params'], cv=5, return_train_score=False, n_jobs=-1)

    clf.fit(X_train, y_train)
    scores.append({
        'model': algo,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    best_estimators[algo] = clf.best_estimator_
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.801605,"{'svc__C': 1, 'svc__kernel': 'linear'}"
1,logistic,0.810598,{'logisticregression__C': 5}


In [9]:
best_estimators['svm'].score(X_test,y_test)

0.8201438848920863

In [10]:
best_estimators['logistic'].score(X_test,y_test)

0.8273381294964028

In [11]:
df['best_params'][0]

{'svc__C': 1, 'svc__kernel': 'linear'}

In [12]:
best_estimators

{'svm': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('svc', SVC(C=1, kernel='linear', probability=True))]),
 'logistic': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(C=5))])}

In [ ]:
y_pred = best_estimators['svm'].predict(X_test)

In [14]:
le.inverse_transform(y_pred)

array(['Rishi Sunak', 'Donald Trump', 'Xi Jinping', 'Volodymyr Zelenskyy',
       'Narendra Modi', 'Arvind Kejriwal', 'Joe Biden',
       'YS Rajashekar Reddy', 'ChandraBabu Naidu', 'Vladimir Putin',
       'Angela Merkel', 'YS Jagan', 'Jacinda Ardern', 'Rahul Gandhi',
       'Rishi Sunak', 'Donald Trump', 'Angela Merkel', 'YS Jagan',
       'Xi Jinping', 'Rahul Gandhi', 'YS Rajashekar Reddy',
       'Rahul Gandhi', 'Joe Biden', 'Narendra Modi', 'YS Jagan',
       'Narendra Modi', 'Olaf Scholz', 'Boris Johnson', 'Narendra Modi',
       'Scott Morrison', 'Olaf Scholz', 'Vladimir Putin',
       'Arvind Kejriwal', 'Justin Trudeau', 'Rishi Sunak',
       'Jacinda Ardern', 'Narendra Modi', 'Volodymyr Zelenskyy',
       'Emmanuel Macron', 'YS Jagan', 'Rishi Sunak', 'Boris Johnson',
       'YS Rajashekar Reddy', 'Justin Trudeau', 'Olaf Scholz',
       'Narendra Modi', 'ChandraBabu Naidu', 'Angela Merkel',
       'Boris Johnson', 'Narendra Modi', 'Joe Biden', 'Xi Jinping',
       'Volodymyr Ze

In [15]:
best_estimators['svm'].predict_proba(X_test)[0]

array([5.80191913e-04, 8.84991457e-04, 6.62437556e-04, 5.04079399e-03,
       3.93317179e-04, 1.10075519e-03, 4.53096457e-04, 3.94800978e-04,
       2.99910363e-04, 3.58580170e-03, 2.83178486e-04, 7.85968755e-04,
       2.76524271e-03, 9.76731674e-01, 3.37058074e-04, 3.25301835e-04,
       1.48510666e-03, 2.49819139e-04, 2.36360088e-03, 1.27695234e-03])

In [16]:
import joblib 

joblib.dump(best_estimators['svm'], 'model.pkl')

['model.pkl']

In [8]:
import json 
classes = le.classes_.tolist()
classes_dict = {classes[i]:i for i in range(len(classes))}
with open('classes.json','w') as f:
    f.write(json.dumps(classes_dict))

In [7]:
le.classes_.tolist()

['Angela Merkel',
 'Arvind Kejriwal',
 'Boris Johnson',
 'ChandraBabu Naidu',
 'Donald Trump',
 'Emmanuel Macron',
 'Jacinda Ardern',
 'Joe Biden',
 'Justin Trudeau',
 'Nara Lokesh',
 'Narendra Modi',
 'Olaf Scholz',
 'Rahul Gandhi',
 'Rishi Sunak',
 'Scott Morrison',
 'Vladimir Putin',
 'Volodymyr Zelenskyy',
 'Xi Jinping',
 'YS Jagan',
 'YS Rajashekar Reddy']

In [7]:
len(X_raw[0])

8100

In [11]:
len(X_reduced[0])

100